In [ ]:
# Lookalike Model
#This notebook demonstrates the development of a Lookalike Model to recommend similar customers based on their spending patterns, purchase quantities, and product diversity.


In [ ]:
## Data Preparation
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Load datasets
customers = pd.read_csv('data/Customers.csv')
transactions = pd.read_csv('data/Transactions.csv')

# Merge and aggregate data
customer_agg = customers[['CustomerID']].copy()
transaction_agg = transactions.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'Quantity': 'sum',
    'ProductID': 'nunique'
}).reset_index()

transaction_agg.rename(columns={'ProductID': 'NumProductsPurchased'}, inplace=True)

# Merge with customer data
customer_agg = customer_agg.merge(transaction_agg, on='CustomerID', how='left')
customer_agg.fillna(0, inplace=True)

display(customer_agg.head())


In [ ]:
## Computing Cosine Similarity
# Normalize data
scaler = StandardScaler()
features = ['TotalValue', 'Quantity', 'NumProductsPurchased']
customer_agg_scaled = scaler.fit_transform(customer_agg[features])

# Compute cosine similarity
similarity_matrix = cosine_similarity(customer_agg_scaled)
similarity_df = pd.DataFrame(similarity_matrix, index=customer_agg['CustomerID'], columns=customer_agg['CustomerID'])

display(similarity_df.head())


In [ ]:
## Generate Recommendations
def get_top_3_lookalikes(customer_id, similarity_df):
    similar_customers = similarity_df.loc[customer_id].sort_values(ascending=False).iloc[1:4]
    return similar_customers

# Generate recommendations for first 20 customers
recommendations = {}
for customer_id in customer_agg['CustomerID'].head(20):
    top_3 = get_top_3_lookalikes(customer_id, similarity_df)
    recommendations[customer_id] = list(top_3.index)

display(recommendations)
